Code to predict probability of credit card defaulting based on UCL data

Making necessary imports

In [1]:
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

f:\python\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Reading excel file and converting to CSV

In [2]:
df = pd.read_excel("./Credit_Card_Data.xlsx")
df.to_csv("./UCL_Credit_Card_data.csv", sep=",",header = ['ID','LIMIT_BAL','SEX','EDUCATION','MARRIAGE','AGE','PAYMENT_STATUS'])

In [3]:
print(df.head)

<bound method NDFrame.head of           ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  \
0          1      20000    2          2         1   24   
1          2     120000    2          2         2   26   
2          3      90000    2          2         2   34   
3          4      50000    2          2         1   37   
4          5      50000    1          2         1   57   
5          6      50000    1          1         2   37   
6          7     500000    1          1         2   29   
7          8     100000    2          2         2   23   
8          9     140000    2          3         1   28   
9         10      20000    1          3         2   35   
10        11     200000    2          3         2   34   
11        12     260000    2          1         2   51   
12        13     630000    2          2         2   41   
13        14      70000    1          2         2   30   
14        15     250000    1          1         2   29   
15        16      50000    2          3   

Splitting test and train data

In [4]:

df['split'] = np.random.randn(df.shape[0], 1)

msk = np.random.rand(len(df)) <= 0.7

train = df[msk]
test = df[~msk]

x_train = train.iloc[:,1:6]
y_train = train.iloc[:,6:7]
x_test = train.iloc[:,1:6]
y_test = train.iloc[:,6:7]

Creating our model

In [5]:
clf = LogisticRegression()
clf.fit(x_train, y_train.values.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Accuracy computation

In [6]:
accuracy = clf.score(x_test, y_test)
print(accuracy)

0.7762674679257882
